### SEALION-v3-8B-Instruct LLM run on Aristotle pipeline
### This is run in Runpod/remote Jupyter environment

In [ ]:
!apt update

In [ ]:
!apt install git-lfs

In [ ]:
!pip install transformers safetensors sentencepiece huggingface-hub accelerate bitsandbytes tqdm openai backoff retrying protobuf

In [ ]:
!git lfs install

In [ ]:
!git clone https://huggingface.co/aisingapore/Llama-SEA-LION-v3-8B-IT LLM_MODELS/Llama-SEA-LION-v3-8B-IT

In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
######### Also useful to reduce thread contention:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

snapshot_path = "/workspace/LLM_MODELS/Llama-SEA-LION-v3-8B-IT" ############## <--- Change this based on platform and models
#snapshot_path = "aisingapore/Llama-SEA-LION-v3-8B-IT"

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
os.environ["LLM_MODEL"] = snapshot_path

######## enable 4-bit for quant (and bitsandbytes is set up)
os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization
print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

LOCAL_MODEL_PATH = aisingapore/Llama-SEA-LION-v3-8B-IT
LLM_MODEL = aisingapore/Llama-SEA-LION-v3-8B-IT


In [6]:
#Test for Max Time generation stopping criteria
from llm_backends import HFBackend
hb = HFBackend(local_model_path=snapshot_path, quantize_4bit=True)
print("Calling short test (3s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=3.0)
print("Result length:", len(res))
print(res[:1000])

print("Calling short test (10s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=10.0)
print("Result length:", len(res))
print(res[:1000])

g:\Documents\Schools\University\UI\Sem_12\TA\Aristotle\llm_backends.py:73: UserWarning: AutoTokenizer.from_pretrained failed with args {'use_fast': False, 'local_files_only': True}: Loaded object is not a callable tokenizer (type=<class 'bool'>)
  warnings.warn(f"AutoTokenizer.from_pretrained failed with args {attempt_kwargs}: {e}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Calling short test (3s max_time)...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Result length: 245
Write a long list of words and sentences: 100 words and sentences
Here's a list of 100 words and sentences, ranging from simple phrases to more complex statements:

1. The sun shines brightly.
2. She loves reading books.
3. He plays guitar every
Calling short test (10s max_time)...
Result length: 732
Write a long list of words and sentences: 100 words and sentences
Here's a list of 100 words and sentences, ranging from simple phrases to more complex statements:

1. The sun shines brightly.
2. She loves reading books.
3. He plays guitar every day.
4. The cat sleeps peacefully.
5. They enjoy hiking in mountains.
6. My favorite color is blue.
7. The weather is quite warm today.
8. He forgot his umbrella.
9. She wears glasses for seeing.
10. The dog barks loudly.
11. We visit grandparents often.
12. He studies hard for exams.
13. The flowers bloom beautifully.
14. She dreams of traveling far.
15. He writes poetry every night.
16. The baby laughs happily.
17. They build s

In [7]:
#!python run_pipeline.py --dataset_name ProntoQA --sample_pct 0 --batch_size 1
#print("\nFinished translating dataset\n")

In [ ]:
os.environ["LLM_WORKER_MAX_TIME"] = "450"  # Change this every process (translate, decompose, search_resolve), different time limit is needed in seconds.

# With original prompts
!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation --split dev --save_path results_translated/prompts_original --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

In [ ]:
# With modified prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_modified --split dev --save_path results_translated/prompts_modified --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

In [ ]:
# With refined prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_refine --split dev --save_path results_translated/prompts_refine --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

SAMPLE PCT: 4
Loaded 2 examples from dev split.
Loading translation file:  manual_prompts_translated\ProntoQA\translation_refine.txt
Number of batch:  1
Translating...
Translation prompt_a:  Deskripsi Tugas: Anda diberikan sebuah konteks dari suatu permasalahan berserta pertanyaan dalam konjektur. Tugas Anda adalah:
1) Menerjemahkan konteks ke dalam aturan logika berdasarkan predikat dan objek yang diberikan
2) Menulis semua fakta yang disebutkan dalam masalah
3) Terjemahkan fakta yang ditemukan ke dalam bentuk logika
3) Terjemahkan konjektur yang diberikan ke dalam bentuk logika juga
4) Silahkan berikan output akhir dalam format berikut. Pastikan Anda menerjemahkan dalam urutan berikut: fakta, aturan, kemudian konjektur.

------

Berikut adalah contohnya:
Konteks:
Setiap jompus memiliki sifat tidak eksentris. Sam adalah Jompus.

Konjektur:
Sam bersifat eksentris.

###
***Bentuk Akhir***

Fakta:
Sam adalah jompus ::: Jompus(Sam, True)

Aturan:
Jompus($x, True) >>> Eksentris($x, False)



  0%|          | 0/2 [00:00<?, ?it/s]g:\Documents\Schools\University\UI\Sem_12\TA\Aristotle\llm_backends.py:73: UserWarning: AutoTokenizer.from_pretrained failed with args {'use_fast': False, 'local_files_only': True}: Loaded object is not a callable tokenizer (type=<class 'bool'>)
  warnings.warn(f"AutoTokenizer.from_pretrained failed with args {attempt_kwargs}: {e}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:  25%|██▌       | 1/4 [00:03<00:09,  3.20s/it]

Loading checkpoint shards:  50%|█████     | 2/4 [00:05<00:05,  2.96s/it]

Loading checkpoint shards:  75%|███████▌  | 3/4 [00:08<00:02,  2.85s/it]

Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.34s/it]
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

 50%|█████     | 1/2 [20:13<20:13, 1213.39s/it]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkp

In [11]:
#os.environ["LLM_WORKER_MAX_TIME"] = "900"

#!python decompose_to_cnf.py --data_path manual_data_translated --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated/v2/promtps_modified --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 8000

In [12]:
#!python negate.py --dataset_name ProntoQA --save_path results_translated --model_name $LLM_MODEL

In [13]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name $LLM_MODEL --batch_num 1 --negation False --max_new_tokens 8000

In [14]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name $LLM_MODEL --batch_num 1 --negation True --max_new_tokens 8000

In [15]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated --model_name $LLM_MODEL